In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In this notebook, we will run a ready-made network starting from some ATLAS data, which is already normalized. There is also an alternative to train the network from scratch.  

## Look into the dataset

First we need to make sure that Python 3.8 is used in the notebook. It is required in order to open this certain .pkl-file. 

In [ ]:
import sys
sys.version

We take a pickle dataset, and open into Pandas (after importing pandas). Note that you have to change the paths to the directory where your processed files are. 

In [ ]:
import pandas as pd

# Change these paths to point to where you have stored the datasets. 
data_path = '/content/monojet_Zp2000.0_DM_50.0_chan3.csv'
preprocessed_path = '/content/preprocessed.csv'

# Data is separated by ; for event details
# Specific event object information is separated by ,
# This means no specific length limit for the rows
# From https://stackoverflow.com/questions/55188544/pandas-how-to-workaround-error-tokenizing-data
data = pd.read_csv(data_path, sep='\n', header=None)[0].str.split(';', expand=True)

# The final state objects, as described in Table 1, are stored in a one-line-per-event text (csv) file,
# where each line has variable length and contains 3 event-specifiers, followed by the kinematic fea-
# tures for each object in the event. The format of CSV files is:
# event ID; process ID; event weight; MET; METphi; obj1, E1, pt1, eta1, phi1; obj2, E2, pt2, eta2, phi2; . . .

# TLDR: Strip out the unnecessary fields, not a 1 to 1 compression but data compression of relevant objects
# Your task is to compress the "four-momentum" of the jet particles that are described as follows in the .csv file:
# obj1, E1, pt1, eta1, phi1
# You should select only one kind of particle - the jets (id = "j") - and compress their four-momentum using a network 
# that is similar (or the same) to the one in this Jupyter notebook:

# Strip out the eventID, process ID, event weight, MET, METphi
data_columns = data.columns
relevant_data = data[range(5, len(data_columns))]

# Not quite ideal but we can reconvert to CSV to take advantage of some features
relevant_data.to_csv(preprocessed_path, sep=';', index=False, header=False)

In [ ]:
import numpy as np

columns = ["type","E","pt","eta","phi"] # m is missing 

# Reads the data through Pandas
train = pd.read_csv(preprocessed_path, sep=',', lineterminator=';', names=columns)
test = pd.read_csv(preprocessed_path, sep=',', lineterminator=';', names=columns)

# We only want to compress a single type of object
train = train[train["type"] == 'j'][columns[1:]]
test = test[test["type"] == 'j'][columns[1:]]

# Normalize data as expected
def normalize_data(df):
  df["eta"] /= 5
  df["phi"] /= 3
  df["E"] = np.log(df["E"]) / np.log(10)
  df["pt"] = np.log(df["pt"]) / np.log(10)
  return df

train = normalize_data(train)
test = normalize_data(test)

# To get an idea of the order of magnitude we are going to see in the plots we show the first elements 
# in the samples:
print('Training sample:')
print(train.head())

print('\n')

print('Testing sample:')
print(test.head())

print('\n')

print('The number of entries in the training data:', len(train))
print('The number of entries in the validation data:', len(test))

Now we plot the data using the matplotlib library. The units reflect the normalization, but it's the shape that we care about. 

In [ ]:
import matplotlib.pyplot as plt

unit_list = ['[log(GeV)]', '[rad/3]', '[rad/3]', '[log(GeV)]']
variable_list = [r'$E$', r'$p_T$', r'$\eta$', r'$\phi$']

branches=["E","pt","eta","phi"] # No m

n_bins = 100

for kk in range(0,len(branches)):
    n_hist_data, bin_edges, _ = plt.hist(train[branches[kk]], color='gray', label='Input', alpha=1, bins=n_bins)
    plt.xlabel(xlabel=variable_list[kk] + ' ' + unit_list[kk])
    plt.ylabel('# of events')
    plt.savefig("fourmomentum_"+branches[kk],dpi=300)
    plt.show()

## Setting up the network

### Preparing the data

Adding the two datasets as TensorDatasets to PyTorch (also loading all other classes we'll need later)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from fastai import learner
from fastai.data import core

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

# m seems to some clutch thing going on
train_x = train
test_x = test
train_y = train_x  # y = x since we are building an autoencoder
test_y = test_x

# Constructs a tensor object of the data and wraps them in a TensorDataset object.
train_ds = TensorDataset(torch.tensor(train_x.values, dtype=torch.float).to(device), torch.tensor(train_y.values, dtype=torch.float).to(device))
valid_ds = TensorDataset(torch.tensor(test_x.values, dtype=torch.float).to(device), torch.tensor(test_y.values, dtype=torch.float).to(device))

We now set things up to load the data, and we use a batch size that was optimized by previous students...note also that this is fastai v2, migration thanks to Jessica Lastow.

In [ ]:
bs = 256

# Converts the TensorDataset into a DataLoader object and combines into one DataLoaders object (a basic wrapper
# around several DataLoader objects). 
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
dls = core.DataLoaders(train_dl, valid_dl)

### Preparing the network

Here we have an example network. Details aren't too important, as long as they match what was already trained for us...in this case we have a LeakyReLU, tanh activation function, and a number of layers that goes from 4 to 200 to 20 to 3 (number of features in the hidden layer that we pick for testing compression) and then back all the way to 4. 

In [ ]:
class AE_3D_200(nn.Module):
  def __init__(self, input_features, output_features):
    super(AE_3D_200, self).__init__()
    # Encoder network
    self.encoder = nn.Sequential(
        nn.Linear(input_features, 200),
        nn.Tanh(),
        nn.Linear(200, 200),
        nn.Tanh(),
        nn.Linear(200, 20),
        nn.Tanh(),
        nn.Linear(20, output_features),
    )
    # Decoder network
    self.decoder = nn.Sequential(
        nn.Linear(output_features, 20),
        nn.Tanh(),
        nn.Linear(20, 200),
        nn.Tanh(),
        nn.Linear(200, 200),
        nn.Tanh(),
        nn.Linear(200, input_features),
    )

  def encode(self, x):
    return self.encoder(x)

  def decode(self, x):
    return self.decoder(x)

  def forward(self, x):
    return self.decode(self.encode(x))

  def describe(self):
    return 'AE-200-20'

model = AE_3D_200(4, 3)
model.to(device)

We now have to pick a loss function - MSE loss is appropriate for a compression autoencoder since it reflects the [(input-output)/input] physical quantity that we want to minimize. 

In [ ]:
from fastai.metrics import mse

loss_func = nn.MSELoss()

#bn_wd = False  # Don't use weight decay for batchnorm layers
#true_wd = True  # weight decay will be used for all optimizers
wd = 1e-6

recorder = learner.Recorder()
learn = learner.Learner(dls, model=model, wd=wd, loss_func=loss_func, cbs=recorder)
# was learn = basic_train.Learner(data=db, model=model, loss_func=loss_func, wd=wd, callback_fns=ActivationStats, bn_wd=bn_wd, true_wd=true_wd)

## Alternative 1: Running a pre-trained network

Now we load the pre-trained network. 

In [ ]:
# learn.load("4D_TLA_leading")

Then we evaluate the MSE on this network - it should be of the order of 0.001 or less if all has gone well...if it has not trained as well (note the pesky 0-mass peak above...) then it's going to be a bit higher.

In [ ]:
# learn.validate()

## Alternative 2: Training a new network

Instead of using a pre-trained network, an alternative is to train a new network and use that instead. 

First, we want to find the best learning rate. The learning rate is a hyper-paramater that sets how much the weights of the network will change each step with respect to the loss gradient.

Then we plot the loss versus the learning rates. We're interested in finding a good order of magnitude of learning rate, so we plot with a log scale.

A good value for the learning rates is then either:
- one tenth of the minimum before the divergence
- when the slope is the steepest

In [ ]:
from fastai.callback import schedule

lr_min, lr_steep = learn.lr_find()

print('Learning rate with the minimum loss:', lr_min)
print('Learning rate with the steepest gradient:', lr_steep)

Now we want to run the training!

User-chosen variables:
- n_epoch: The number of epochs, i.e how many times the to run through all of the training data once (i.e the 1266046 entries, see cell 2)
- lr: The learning rate. Either choose lr_min, lr_steep from above or set your own.


In [ ]:
import time

start = time.perf_counter() # Starts timer
# learn.fit_one_cycle(n_epoch=100, lr=lr_min)
learn.fit_one_cycle(100, slice(lr_steep)) # guess this changed
end = time.perf_counter() # Ends timer
delta_t = end - start
print('Training took', delta_t, 'seconds')

Then we plot the loss as a function of batches and epochs to check if we reach a plateau.

In [ ]:
recorder.plot_loss()

Then we evaluate the MSE on this network - it should be of the order of 0.001 or less if all has gone well...if it has not trained as well (note the pesky 0-mass peak above...) then it's going to be a bit higher.

In [ ]:
learn.validate()

Let's plot all of this, with ratios (thanks to code by Erik Wallin)

## Plotting the outputs of the network

Lazy-save of our output files (they'll also be on screen)

In [ ]:
import os
save_dir = "plotOutput"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

A function in case we want to un-normalize and get back to physical quantities...

In [ ]:
def custom_unnormalize(df):
    df['eta'] = df['eta'] * 5
    df['phi'] = df['phi'] * 3
    df['E'] = 10**df['E']
    # df['m'] = 10**df['m']
    df['pt'] = 10**(df['pt'])
    return df

Make the histograms from the dataset...

In [ ]:
import numpy as np

plt.close('all')
unit_list = ['[GeV]', '[rad]', '[rad]', '[GeV]']
variable_list = [r'$E$', r'$p_T$', r'$\eta$', r'$\phi$']
line_style = ['--', '-']
colors = ['orange', 'c']
markers = ['*', 's']

model.to(device)

save = True # Option to save figure

# Histograms
idxs = (0, 100000)  # Choose events to compare
data = torch.tensor(test[idxs[0]:idxs[1]].values, dtype=torch.float).to(device)
# data = torch.tensor(test[idxs[0]:idxs[1]].values, dtype=torch.float).double().to(device)
pred = model(data)
pred = pred.detach().to('cpu').numpy()
data = data.detach().to('cpu').numpy()

data_df = pd.DataFrame(data, columns=test.columns)
pred_df = pd.DataFrame(pred, columns=test.columns)

# Unnormalizing creates inf values...
unnormalized_data_df = custom_unnormalize(data_df)
unnormalized_pred_df = custom_unnormalize(pred_df)    
    
alph = 0.8
n_bins = 200
for kk in np.arange(4):
    plt.figure()
    n_hist_data, bin_edges, _ = plt.hist(data[:, kk], color=colors[1], label='Input', alpha=1, bins=n_bins)
    n_hist_pred, _, _ = plt.hist(pred[:, kk], color=colors[0], label='Output', alpha=alph, bins=bin_edges)
    plt.suptitle(test.columns[kk])
    plt.xlabel(test.columns[kk])
    plt.ylabel('Number of events')
    # ms.sciy()
    plt.yscale('log')
    plt.legend()
    if save:
        plt.savefig(os.path.join(save_dir,test.columns[kk]+'.png'))

In [ ]:
def getRatio(bin1,bin2):
    bins = []
    for b1,b2 in zip(bin1,bin2):
        if b1==0 and b2==0:
            bins.append(0.)
        elif b2==0:
            bins.append(None)
        else:
            bins.append((float(b2)-float(b1))/b1)
    return bins   

rat = getRatio(n_hist_data,n_hist_pred)

sum = 0.0
for i in rat:
  sum = sum + i

print(sum / len(rat))
print(rat)